In [9]:
!pip install -U imbalanced-learn
!pip install --upgrade pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/d6/7c/20e737300f9bec011fb79c01d8948bc38c854876aac2da2cfcdd0992b153/pandas-2.1.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
import math,timeit
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, fbeta_score
from imblearn.over_sampling import RandomOverSampler,SMOTE

In [3]:
df=pd.read_csv('data10.csv')

In [11]:
df2=pd.read_csv('2018_best_dtypes.csv')

In [12]:
df.head

<bound method NDFrame.head of          Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0            119646801            17            16              597   
1                  375             2             1               38   
2                  281             3             0               46   
3             99999555             2             2               16   
4             10070086             4             4              379   
...                ...           ...           ...              ...   
1033926        1569097             8             7             1148   
1033927        1731968             8             7             1144   
1033928          31979             2             2               62   
1033929       14218439            11             8             1148   
1033930           1044             5             2              935   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
0                    768             

In [13]:
df2.head

<bound method NDFrame.head of              Column    Dtype
0     Flow Duration    int32
1      Tot Fwd Pkts    int32
2      Tot Bwd Pkts    int32
3   TotLen Fwd Pkts    int32
4   TotLen Bwd Pkts    int32
..              ...      ...
64        Idle Mean  float32
65         Idle Std  float32
66         Idle Max    int32
67         Idle Min    int32
68            Label   object

[69 rows x 2 columns]>

In [21]:
df.columns

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts',
       'Subflow F

In [24]:
df2.loc[68].Column='category'

In [25]:
df2.loc[68]

Column    category
Dtype       object
Name: 68, dtype: object

In [20]:
df.columns.size
df['category']

0          0
1          0
2          0
3          4
4          0
          ..
1033926    0
1033927    0
1033928    0
1033929    0
1033930    0
Name: category, Length: 1033931, dtype: int64

In [17]:
X = df.drop('category', axis=1)
y = df['category']

# Step 4: Divide data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


## 1)Equal no. of benign and attack records after over-sampling, proportion of different attack class records remain same

In [30]:
Counter(y)
sampling_strategy = round((y.value_counts()[0]/y.value_counts()[1:].sum())*y.value_counts()[1:]).to_dict()
sampling_strategy = {key: int(value) for key, value in sampling_strategy.items()}
sampling_strategy


{3: 527515, 4: 133634, 1: 98256, 5: 77378, 2: 63972, 6: 591}

### Oversampling Attack Records

In [31]:
rus = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_undersampled, y_undersampled = rus.fit_resample(X, list(y))
y_undersampled = pd.Series(y_undersampled)
Counter(y_undersampled)
y_undersampled.value_counts()[1:].sum()


901346

In [32]:
sampling_strategy = {0: 901347, 1: 150224, 2: 150224, 3: 150224, 4: 150224, 5: 150224, 6: 150224}
rus = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, list(y))

In [34]:
print("Class distribution after oversampling:")
y_resampled = pd.Series(y_resampled)
y_resampled.value_counts()[1:].sum()

Class distribution after oversampling:


901344

#### Splitting data in training and testing

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

In [37]:
# Value counts of category after splitting
y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

In [39]:
##Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
##PCA
pca = PCA(n_components=0.95, random_state=0)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
##Variance ratios
variance_ratios = pca.explained_variance_ratio_
variance_ratios

array([0.41858996, 0.17650799, 0.11901362, 0.06332868, 0.0512147 ,
       0.04994341, 0.03179182, 0.02265315, 0.01893804])

In [40]:
# converting the arrays given by PCA to DataFrame
X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca=pd.DataFrame(X_test_pca)

In [42]:
##Random Forest Classifier
model = RandomForestClassifier()
# removing non-integer values from labels 
y_train=y_train.astype('int')
y_test=y_test.astype('int')


In [43]:
## Calculating Training Time
training_time = timeit.timeit(lambda: model.fit(X_train_pca, y_train), number=1) 
start_time = timeit.default_timer()
y_pred = model.predict(X_test_pca)
testing_time = timeit.default_timer() - start_time
print("Training Time:", training_time) 
print("Testing Time:", testing_time)

Training Time: 344.6587489169906
Testing Time: 6.878465958987363


In [45]:
## Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[266055,     17,      0,     31,     17,   4147,    138],
       [     0,  45058,      0,      0,      0,     10,      0],
       [     0,      0,  45067,      0,      0,      0,      0],
       [     8,      0,      0,  45059,      0,      0,      0],
       [     0,      0,      0,      0,  45067,      0,      0],
       [  1636,      4,      0,      0,      0,  43427,      0],
       [     0,      0,      0,      0,      0,      0,  45067]])

In [46]:
## Accuracy
accuracy_score(y_test, y_pred)

0.9888906968831822

In [47]:
## Precision
precision_score(y_test, y_pred, average='weighted')

0.9892673903827923

In [48]:
## Recall
recall_score(y_test, y_pred, average='weighted')

0.9888906968831822

In [49]:
## F2 Score
fbeta_score(y_test, y_pred, beta=2, average='weighted')


0.9889203895406997

## 2)Equal no. of benign and attacks records, no. of records of different attack classes must be equal

In [50]:
# samples of each to make the sum of attack records equal to Bengin
count = math.ceil(y.value_counts()[0]/6)
count

150225

In [51]:
count_of_benign = count*6

In [52]:
sampling_strategy = {0: count_of_benign}
for i in range(1, 7): sampling_strategy[i] = count

### Oversampling Every Label to Match the Sum of Bengin and Attack Records

In [53]:
rus = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, list(y))
y_resampled = pd.Series(y_resampled)
y_resampled.value_counts()

0    901350
4    150225
3    150225
1    150225
5    150225
2    150225
6    150225
Name: count, dtype: int64

In [54]:
## Splitting data in training and testing 
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

In [55]:
## Value Counts after splitting
y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

In [56]:
##Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
##PCA
pca = PCA(n_components=0.95, random_state=0)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
##Variance ratios
variance_ratios = pca.explained_variance_ratio_
variance_ratios

array([0.41297372, 0.17085201, 0.11566945, 0.06194299, 0.05379848,
       0.04876927, 0.03412318, 0.02764539, 0.01904806, 0.01157388])

In [57]:
# converting the arrays given by PCA to DataFrame
X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca=pd.DataFrame(X_test_pca)

In [58]:
##Random Forest Classifier
model = RandomForestClassifier()
# removing non-integer values from labels 
y_train=y_train.astype('int')
y_test=y_test.astype('int')


In [59]:
## Calculating Training Time
training_time = timeit.timeit(lambda: model.fit(X_train_pca, y_train), number=1) 
start_time = timeit.default_timer()
y_pred = model.predict(X_test_pca)
testing_time = timeit.default_timer() - start_time
print("Training Time:", training_time) 
print("Testing Time:", testing_time)

Training Time: 350.95075970800826
Testing Time: 6.8974762500147335


In [60]:
## Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[266085,     22,      1,     16,     16,   4126,    139],
       [     0,  45068,      0,      0,      0,      0,      0],
       [     0,      0,  45067,      0,      0,      0,      0],
       [    14,      0,      0,  45052,      0,      2,      0],
       [     0,      0,      0,      0,  45068,      0,      0],
       [  1620,     13,      0,      0,      0,  43434,      0],
       [     0,      0,      0,      0,      0,      0,  45067]])

In [61]:
## Accuracy
accuracy_score(y_test, y_pred)

0.988962852018269

In [62]:
## Precision
precision_score(y_test, y_pred, average='weighted')

0.9893338001711459

In [63]:
## Recall
recall_score(y_test, y_pred, average='weighted')

0.988962852018269

In [64]:
## F2 Score
fbeta_score(y_test, y_pred, beta=2, average='weighted')


0.9889921344695091